In [1]:
# Install required libraries (Colab friendly)
!pip install pandas plotly


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


In [3]:
# Load datasets
sales_df = pd.read_csv("/content/sales_data.csv")
customer_df = pd.read_csv("/content/customer_churn.csv")

# Preview
sales_df.head(), customer_df.head()


(         Date     Product  Quantity  Price Customer_ID Region  Total_Sales
 0  2024-01-01       Phone         7  37300     CUST001   East       261100
 1  2024-01-02  Headphones         4  15406     CUST002  North        61624
 2  2024-01-03       Phone         2  21746     CUST003   West        43492
 3  2024-01-04  Headphones         1  30895     CUST004   East        30895
 4  2024-01-05      Laptop         8  39835     CUST005  North       318680,
   CustomerID  Tenure  MonthlyCharges  TotalCharges        Contract  \
 0     C00001       6              64          1540        One year   
 1     C00002      21             113          1753  Month-to-month   
 2     C00003      27              31          1455        Two year   
 3     C00004      53              29          7150  Month-to-month   
 4     C00005      16             185          1023        One year   
 
       PaymentMethod PaperlessBilling  SeniorCitizen  Churn  
 0       Credit Card               No              1 

In [4]:
# Basic info
print(sales_df.info())
print(customer_df.info())

# Check missing values
print("\nSales Missing Values:\n", sales_df.isnull().sum())
print("\nCustomer Missing Values:\n", customer_df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         100 non-null    object
 1   Product      100 non-null    object
 2   Quantity     100 non-null    int64 
 3   Price        100 non-null    int64 
 4   Customer_ID  100 non-null    object
 5   Region       100 non-null    object
 6   Total_Sales  100 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 5.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CustomerID        500 non-null    object
 1   Tenure            500 non-null    int64 
 2   MonthlyCharges    500 non-null    int64 
 3   TotalCharges      500 non-null    int64 
 4   Contract          500 non-null    object
 5   PaymentMethod     500 non-null    object
 

In [5]:
# Convert date column
sales_df['Date'] = pd.to_datetime(sales_df['Date'])

# Extract date parts
sales_df['Year'] = sales_df['Date'].dt.year
sales_df['Month'] = sales_df['Date'].dt.month
sales_df['Month_Name'] = sales_df['Date'].dt.month_name()

# Clean text columns
sales_df['Product'] = sales_df['Product'].str.strip().str.title()
sales_df['Region'] = sales_df['Region'].str.upper()

sales_df.head()


,Date,Product,Quantity,Price,Customer_ID,Region,Total_Sales,Year,Month,Month_Name
0,2024-01-01,Phone,7,37300,CUST001,EAST,261100,2024,1,January
1,2024-01-02,Headphones,4,15406,CUST002,NORTH,61624,2024,1,January
2,2024-01-03,Phone,2,21746,CUST003,WEST,43492,2024,1,January
3,2024-01-04,Headphones,1,30895,CUST004,EAST,30895,2024,1,January
4,2024-01-05,Laptop,8,39835,CUST005,NORTH,318680,2024,1,January


In [6]:
# Preprocess Customer IDs to ensure compatibility for merging
sales_df['Customer_ID_cleaned'] = sales_df['Customer_ID'].str.replace('CUST', '', regex=False).astype(int)
customer_df['CustomerID_cleaned'] = customer_df['CustomerID'].str.replace('C', '', regex=False).astype(int)

# Merge datasets using the cleaned Customer IDs
merged_df = pd.merge(
    sales_df,
    customer_df,
    left_on="Customer_ID_cleaned",
    right_on="CustomerID_cleaned",
    how="inner"
)

# Drop the temporary cleaned ID columns if no longer needed, or keep for consistency
# merged_df = merged_df.drop(columns=['Customer_ID_cleaned', 'CustomerID_cleaned'])

merged_df.head()

,Date,Product,Quantity,Price,Customer_ID,Region,Total_Sales,Year,Month,Month_Name,...,CustomerID,Tenure,MonthlyCharges,TotalCharges,Contract,PaymentMethod,PaperlessBilling,SeniorCitizen,Churn,CustomerID_cleaned
0,2024-01-01,Phone,7,37300,CUST001,EAST,261100,2024,1,January,...,C00001,6,64,1540,One year,Credit Card,No,1,0,1
1,2024-01-02,Headphones,4,15406,CUST002,NORTH,61624,2024,1,January,...,C00002,21,113,1753,Month-to-month,Electronic Check,Yes,1,0,2
2,2024-01-03,Phone,2,21746,CUST003,WEST,43492,2024,1,January,...,C00003,27,31,1455,Two year,Credit Card,No,1,0,3
3,2024-01-04,Headphones,1,30895,CUST004,EAST,30895,2024,1,January,...,C00004,53,29,7150,Month-to-month,Electronic Check,No,1,0,4
4,2024-01-05,Laptop,8,39835,CUST005,NORTH,318680,2024,1,January,...,C00005,16,185,1023,One year,Electronic Check,No,1,0,5


In [7]:
# Customer Lifetime Value
customer_revenue = (
    merged_df.groupby("Customer_ID_cleaned")["Total_Sales"]
    .sum()
    .reset_index()
    .rename(columns={'Customer_ID_cleaned': 'Customer_ID', 'Total_Sales': 'Revenue'})
    .sort_values(by="Revenue", ascending=False)
)

customer_revenue.head()

,Customer_ID,Revenue
15,16,373932
6,7,363870
82,83,350888
72,73,349510
19,20,333992


In [8]:
fig = px.bar(
    customer_revenue.head(10),
    x="Customer_ID",
    y="Revenue",
    title="Top 10 Customers by Revenue",
    labels={"Revenue": "Total Revenue", "Customer_ID": "Customer ID"}
)
fig.show()

In [9]:
monthly_sales = (
    merged_df.groupby(["Year", "Month_Name"])["Total_Sales"]
    .sum()
    .reset_index()
    .rename(columns={'Total_Sales': 'Revenue'}) # Rename to Revenue for consistency after aggregation
)

monthly_sales

,Year,Month_Name,Revenue
0,2024,April,1103468
1,2024,February,2656050
2,2024,January,4120524
3,2024,March,4485006


In [10]:
fig = px.line(
    monthly_sales,
    x="Month_Name",
    y="Revenue",
    color="Year",
    markers=True,
    title="Monthly Sales Trend"
)
fig.show()


In [11]:
pivot_table = pd.pivot_table(
    merged_df,
    values="Total_Sales",
    index="Product",
    columns="Region",
    aggfunc="sum"
)

pivot_table

Region,EAST,NORTH,SOUTH,WEST
Product,,,,
Headphones,288361,107091,512168,476413
Laptop,221946,1798206,1373120,495938
Monitor,642870,397100,39924,268177
Phone,506828,489284,1471428,391854
Tablet,859634,1191954,341212,491540


In [12]:
fig = px.imshow(
    pivot_table,
    text_auto=True,
    title="Revenue Heatmap: Product vs Region"
)
fig.show()


In [13]:
# High value sales in specific region
filtered_df = merged_df[
    (merged_df["Total_Sales"] > 5000) &
    (merged_df["Region"] == "NORTH")
]

filtered_df.head()

,Date,Product,Quantity,Price,Customer_ID,Region,Total_Sales,Year,Month,Month_Name,...,CustomerID,Tenure,MonthlyCharges,TotalCharges,Contract,PaymentMethod,PaperlessBilling,SeniorCitizen,Churn,CustomerID_cleaned
1,2024-01-02,Headphones,4,15406,CUST002,NORTH,61624,2024,1,January,...,C00002,21,113,1753,Month-to-month,Electronic Check,Yes,1,0,2
4,2024-01-05,Laptop,8,39835,CUST005,NORTH,318680,2024,1,January,...,C00005,16,185,1023,One year,Electronic Check,No,1,0,5
8,2024-01-09,Tablet,3,32791,CUST009,NORTH,98373,2024,1,January,...,C00009,64,124,7973,Month-to-month,Credit Card,No,1,0,9
17,2024-01-18,Tablet,4,19322,CUST018,NORTH,77288,2024,1,January,...,C00018,42,101,3736,One year,Bank Transfer,Yes,0,0,18
19,2024-01-20,Laptop,8,41749,CUST020,NORTH,333992,2024,1,January,...,C00020,26,105,6868,Month-to-month,Bank Transfer,Yes,1,0,20


In [14]:
total_revenue = merged_df["Total_Sales"].sum()
total_customers = merged_df["CustomerID"].nunique()
avg_order_value = merged_df["Total_Sales"].mean()
top_customer = customer_revenue.iloc[0]

print("CUSTOMER SALES ANALYSIS REPORT")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Customers: {total_customers}")
print(f"Average Order Value: ${avg_order_value:,.2f}")
print(f"Top Customer: {top_customer['Customer_ID']} with Revenue: ${top_customer['Revenue']:,.2f}")

CUSTOMER SALES ANALYSIS REPORT
Total Revenue: $12,365,048.00
Total Customers: 100
Average Order Value: $123,650.48
Top Customer: 16 with Revenue: $373,932.00
